In [10]:
#import libraries
import cv2
import numpy as np
import imutils
from PIL import Image, ImageFilter
from PIL import ImageEnhance

def change_saturation_opencv(img_path,saturation_param):
    image = cv2.imread(img_path)
    image = np.array(image, dtype=np.uint8)
    hsvImg = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    #multiple by a saturation_param to change the saturation
    hsvImg[...,1] = hsvImg[...,1]*saturation_param
    image=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2BGR)
    cv2.imshow("after",image)

def change_saturation_PIL(img_path,saturation_param1,saturation_param2,saturation_param3):
    im = Image.open( img_path )
    contrast = ImageEnhance.Contrast(im)
    im.show()
    contrast.enhance(saturation_param1).show()
    contrast.enhance(saturation_param2).show()
    contrast.enhance(saturation_param3).show()

def is_contour_bad(c):
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    # the contour is bad if it is a rectangle or traingle or not complex shapes with less than 9 arclength
    return not (len(approx) == 4 or len(approx) == 3 or len(approx) >9 )

def detect_cirlces_ellipses(img_path):
    # load the image as gray scale image
    gray = cv2.imread(img_path,0)
    
    #apply edges detection
    edged = cv2.Canny(gray, 50, 100)
    # find contours in the image and create the mask that will be used to remove the bad contours
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    mask = np.ones(image.shape[:2], dtype="uint8") * 255

    # loop over the contours
    for c in cnts:
        # if the contour is not bad, draw it on the mask
        if is_contour_bad(c):
            cv2.drawContours(mask, [c], -1, 0, -1)

    #now we filtered the simple shapes so its time for detected the circles and ellipses
    #create a kernal for dialtion the noises on the mask
    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(mask,kernel,iterations = 1)
    cv2.imshow("filtered shapes", dilation)
    mask2 = np.ones(image.shape[:2], dtype="uint8") * 255

    #select our target from the filtered shapes
    _, contours, _ = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]
        if len(approx) > 9:
            cv2.drawContours(mask2, [cnt], -1, 0, -1)
    cv2.imshow("circles and ellipses", mask2)
    
def Blob_detection(img_path):
    img = cv2.imread(img_path,0)

    params = cv2.SimpleBlobDetector_Params()


    # Filter by Area.
    params.filterByArea = True
    params.minArea = 150

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.8

    # Filter by Convexity
    params.filterByConvexity = True
    params.minConvexity = 0.8

    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.01

    # Create a detector with the parameters 
    detector = cv2.SimpleBlobDetector_create(params) 

    # Detect blobs 
    keypoints = detector.detect(img) 

    # Draw blobs on our image as red circles 
    blank = np.zeros((1, 1))  
    blobs = cv2.drawKeypoints(img, keypoints, blank, (0, 0, 255), 
                              cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 

    number_of_blobs = len(keypoints) 
    text = "Number of Circular Blobs: " + str(len(keypoints)) 
    cv2.putText(blobs, text, (20, 550), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 255), 2) 

    # Show blobs 
    cv2.imshow("Filtered Circular by Blob ", blobs) 
    
def shapes_with_straight_lines(img_path):
    img = cv2.imread(img_path,0)
    edges = cv2.Canny(img,100,200)

    minLineLength = 400
    maxLineGap = 17
    lines = cv2.HoughLinesP(edges,1,np.pi/80,40,minLineLength,maxLineGap)
    for i in range(1, len(lines)):
        for x1,y1,x2,y2 in lines[i]:
            cv2.line(img,(x1,y1),(x2,y2),(0,255,0),5)

    cv2.imshow("shapes with straigh lines", img)

    
    
#main
img_path='D:/CS/computervision/shapes-25.PNG'
saturation_param=50
saturation_param1=-1
saturation_param2=2
saturation_param3=3

# load the original RGB image
image = cv2.imread(img_path)
#show the loaded image
cv2.imshow("Original", image)

# change saturation
#using opencv
change_saturation_opencv(img_path,saturation_param)
cv2.waitKey(0)
#using PIL
change_saturation_PIL(img_path,saturation_param1,saturation_param2,saturation_param3)
cv2.waitKey(0)
cv2.destroyAllWindows()

#detect circle snd ellipses
#using approxPolyDP
detect_cirlces_ellipses(img_path)
cv2.waitKey(0)
#using BlobDetection
Blob_detection(img_path)
#using HoughLines Transform
cv2.waitKey(0)
shapes_with_straight_lines(img_path)
cv2.waitKey(0)
cv2.destroyAllWindows()